In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import json

# Set up the SPARQL endpoint URL
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

# Set the SPARQL query
query = """
select distinct ?event ?description where {
?event dbp:wikiPageUsesTemplate dbt:France_topics .
?event dbo:abstract ?description .
FILTER(LANG(?description) = "fr").
}
"""

# Set the output format to JSON
sparql.setReturnFormat(JSON)

# Send the SPARQL query to the endpoint
sparql.setQuery(query)
results = sparql.query().convert()

# Save the results as a JSON file
with open("event.json", "w") as f:
    json.dump(results, f, indent=4)


In [ ]:
f = open('event.json')
data = json.load(f)

for d in data['results']['bindings'] :
    print(d['event']['value'])
    print(d['description']['value'])
    print('')
  

In [ ]:
events = []
# affichage des noms et des identifiants des villes
for result in data["results"]["bindings"]:
    name_event = result['event']['value'].split('/')[-1]
    print(name_event)
    events.append({
        "name": name_event,
        "dbpediaUrl": result['event']['value'],
        "description" : result['description']['value']
    })

In [ ]:
with open('event_postTreatment.json', 'w') as outfile:
    json.dump(events, outfile)

In [ ]:
for e in events :
    print(e)

# Essai de retrouver les villes dans les descriptions avec du NLP : ne marche pas dès que c'est pas une ville hyper connue ou Américaine

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

def extract_cities(text):
    doc = nlp(text)
    cities = []
    for ent in doc.ents:
        if ent.label_ == 'GPE': # GPE: geopolitical entity
            cities.append(ent.text)
    return cities


In [ ]:
# Example usage
text = "I love visiting Paris, Berlin and Tokyo."
text2 = "Carloman Ier, né en 751 à Soissons, mort le 4 décembre 771 à Samoussy dans l'Aisne, fut roi des Francs de 768 à 771. Il est le fils et successeur de Pépin le Bref."
cities = extract_cities(text2)
print(cities)  # Output: ['Paris', 'Berlin', 'Tokyo']


# Essai de récupérer les villes à partir du fichier de Yoann

In [ ]:
import json

fcities = open('cities.json')
datac = json.load(fcities)

for d in datac :
    print(d['name'])
    print('')

In [ ]:
villes = []
# affichage des noms et des identifiants des villes
for d in datac:
    villes.append({
        "name": d['name']
    })

for c in villes:
    #print(c['name'])
    #print('')
    if (c["name"] == "Soissons"):
        print(c["name"])

In [ ]:
import re

def find_capital_words(text):
    pattern = r'\b[A-Z][a-z]*\b'  # Matches words beginning with a capital letter
    capital_words = re.findall(pattern, text)
    return capital_words

# Example usage
text = "John Smith is from New York and works at IBM."
capital_words = find_capital_words(text2)
print(capital_words)  # Output: ['John', 'Smith', 'New', 'York', 'IBM']



In [ ]:
capital_words = []
for e in events :
    capital_words.append(find_capital_words(e['description']))
for c in capital_words :
    print(c)


In [ ]:
capital_words = []

for e in events :
    capital_words.append({
        "name": e['name'],
        "capital_words" : (find_capital_words(e['description'])),
        "description" : e['description']
    })
    
for c in capital_words : 
    print(c['name'])
    print(c['capital_words'])
    print(c['description'])
    print('')

In [ ]:
event_to_keep = []

for c in capital_words :
    for i in c['capital_words']:
        for v in villes :   
            if (i== v['name']) and (i!='Francs') :
                event_to_keep.append({
                    "name": c['name'],
                    "description": c['description'],
                    "citie": i
                })



for e in event_to_keep:
    print(e['name'])
    print(e['description'])
    print(e['citie'])
    print('') 

Enlève les doublons

In [ ]:
event_to_keep = [dict(t) for t in {tuple(d.items()) for d in event_to_keep}]
print(unique_list)

In [ ]:
with open('event_to_keep.json', 'w') as outfile:
    json.dump(event_to_keep, outfile, indent=4)